# Recommendation system from scratch

## Import libraries

In [1]:
import time

from numpy import sqrt
from numpy import dot
from numpy.linalg import norm

import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split

sns.set()

## Import data

In [9]:
# Load a rating file
ratings = pd.read_csv('../../recomendation_scratch/data_example/Movielens/ml-latest-small/ratings_test.csv')

In [10]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,5,1,4.0,847434962
2,7,1,4.5,1106635946
3,15,1,2.5,1510577970
4,17,1,4.5,1305696483
...,...,...,...,...
20565,587,910,4.0,953137911
20566,590,910,5.0,1258418246
20567,599,910,2.5,1498516800
20568,603,910,4.0,953926507


In [11]:
X_train, X_test = train_test_split(ratings, test_size=0.2, random_state=42)
print('Train size:', len(X_train))
print('Test size:', len(X_test))

Train size: 16456
Test size: 4114


In [8]:
X_train.userId.value_counts()

5857    101
5954     98
5961     62
5972     58
9325     45
       ... 
6419      2
9341      2
5893      2
4766      2
6786      1
Name: userId, Length: 485, dtype: int64

In [9]:
X_train.movieId.value_counts()

246    69
269    63
417    56
212    52
347    50
       ..
491     1
344     1
219     1
62      1
245     1
Name: movieId, Length: 507, dtype: int64

In [10]:
X_test

,userId,movieId,rating,timestamp
1621,5932,307,10.0,1.458000e+09
2605,2542,412,10.0,1.485994e+09
4890,944,301,6.0,1.543363e+09
2040,5857,228,8.0,1.472083e+09
5399,5885,139,6.6,1.553904e+09
...,...,...,...,...
5365,5972,210,9.0,1.552867e+09
15687,5914,394,8.8,1.679098e+09
1896,5961,128,6.0,1.468109e+09
6694,6786,65,6.3,1.578442e+09


## Collaborative Filtering based on User

In [ ]:
# get rating from the user-item pair
def get_rating(userid,movieid):
    return (ratings.loc[(ratings.userId==userid) & (ratings.movieId == movieid),'rating'].iloc[0])

### Pearson calculation
def pearson_correlation_score(user1, user2):
  both_watch_count = []
  list_movie_user1 = ratings.loc[ratings.userId == user1, 'movieId'].to_list()
  list_movie_user2 = ratings.loc[ratings.userId == user2, 'movieId'].to_list()

  for element in list_movie_user1:
    if element in list_movie_user2:
       both_watch_count.append(element)

  if(len(both_watch_count) == 0):
    return 0;

  rating_sum_1 = sum([get_rating(user1, i) for i in both_watch_count])
  avg_rating_sum_1 = rating_sum_1/len(both_watch_count)

  rating_sum_2 = sum([get_rating(user2, i) for i in both_watch_count])
  avg_rating_sum_2 = rating_sum_2/len(both_watch_count)

  tu = sum([(get_rating(user1, i) - avg_rating_sum_1)*(get_rating(user2, i) - avg_rating_sum_2)  for i in both_watch_count])
  mau = sqrt(sum([pow((get_rating(user1, i) - avg_rating_sum_1),2) for i in both_watch_count]))*sqrt(sum([pow((get_rating(user2, i) - avg_rating_sum_2),2) for i in both_watch_count]))

  if(mau ==0):
    return 0

  return tu/mau

### Cosine calculation
def distance_similarity_score(user1,user2):
    both_watch_count = 0
    for element in ratings.loc[ratings.userId==user1,'movieId'].tolist():
        if element in ratings.loc[ratings.userId==user2,'movieId'].tolist():
            both_watch_count += 1
    if both_watch_count == 0 :
        return 0

    rating1 = []
    rating2 = []
    for element in ratings.loc[ratings.userId==user1,'movieId'].tolist():
        if element in ratings.loc[ratings.userId==user2,'movieId'].tolist():
            rating1.append(get_rating(user1,element))
            rating2.append(get_rating(user2,element))

    return dot(rating1, rating2)/(norm(rating1)*norm(rating2))

### get K user nearest
def most_similar_user(user1, K_number_user, similarity_name):
  userid = ratings.userId.unique().tolist()

  if similarity_name == 'pearson':
    similarity_score =  [(pearson_correlation_score(user1, user_i), user_i) for user_i in userid if user_i != user1]

  if similarity_name == 'cosine':
    similarity_score =  [(distance_similarity_score(user1, user_i), user_i) for user_i in userid if user_i != user1]

  similarity_score.sort() # ascending
  similarity_score.reverse() # descending

  return similarity_score[:K_number_user]

# get items of user's history
def get_movieids_seen(userid):
  return (ratings.loc[(ratings.userId==userid), 'movieId'].tolist())
# Lấy ra tên của một bộ phim
def get_movie_name(movieid):
  return (movies.loc[(movies.movieId==movieid), 'title'].iloc[0])

# get recommendation for 1 user based on Aggregate weighted ratings
def rating_predict_based_on_user(userid, K_number_user, sim_name):
  total = {}
  sum_similarity = {}
  list_user_popular = most_similar_user(userid, K_number_user, sim_name)

  for similarityName, user_pop in list_user_popular: # lặp từng user trong list popular
    score = similarityName
    movieids = get_movieids_seen(user_pop)

    for movieid in movieids:
      if movieid not in get_movieids_seen(userid):
        if movieid not in total:
         total[movieid] = 0
         sum_similarity[movieid] = 0
        # print(f"sum_similarity[{movieid}]_1", sum_similarity[movieid])

        total[movieid] += get_rating(user_pop, movieid)*score
        sum_similarity[movieid] += score
        # print(f"sum_similarity[{movieid}]_2", sum_similarity[movieid])

    # calculate rating according to Aggregate weighted ratings
    list_ranking = []
    for userid,tot in total.items():
     if sum_similarity[userid] == 0:
       list_ranking.append((0,userid))
     else:
       rating = tot/(sum_similarity[userid])
       list_ranking.append((rating,userid))

  list_ranking = [(movieid, tot/sum_similarity[movieid]) for movieid, tot in total.items()]

  return list_ranking

### get name + the high rating predictions of topK=10 items
def get_recommendation_based_on_user(userid, K_number_user, similarity_name, topN = 10):

  list_ranking = rating_predict_based_on_user(userid, K_number_user, similarity_name)

  list_ranking.sort()
  list_ranking.reverse()


  return list_ranking[:topN]

## Collaborative Filtering based on Item

In [ ]:
### Pearson calculation
def pearson_correlation_score_Item(movie1, movie2):
  both_watch_count = [] 
  list_user_movie1 = ratings.loc[ratings.movieId==movie1, 'userId'].to_list()
  list_user_movie2 = ratings.loc[ratings.movieId==movie2, 'userId'].to_list()

  for userid in list_user_movie1:
    if userid in list_user_movie2:
      both_watch_count.append(userid)

  if len(both_watch_count)==0:
    return 0

  sum_rating_1 = sum([get_rating(i, movie1) for i in both_watch_count])
  avg_rating_1 = sum_rating_1/len(both_watch_count) 

  sum_rating_2 = sum([get_rating(i, movie2) for i in both_watch_count])
  avg_rating_2 = sum_rating_2/len(both_watch_count)

  TuSo = sum([(get_rating(i, movie1) - avg_rating_1)*(get_rating(i, movie2) - avg_rating_2) for i in both_watch_count])
  MauSo = sqrt(sum((get_rating(i, movie1) - avg_rating_1)**2 for i in both_watch_count))*sqrt(sum((get_rating(i, movie2) - avg_rating_2)**2 for i in both_watch_count))

  if MauSo==0:
    return 0

  return TuSo/MauSo


### Cosine calculation
def distance_similarity_score_Item(movie1,movie2):

    both_watch_count = 0
    for element in ratings.loc[ratings.movieId==movie1, 'userId'].tolist():
        if element in ratings.loc[ratings.movieId==movie2, 'userId'].tolist():
            both_watch_count += 1
    if both_watch_count == 0 :
        return 0

    rating1 = []
    rating2 = []
    for element in ratings.loc[ratings.movieId==movie1, 'userId'].tolist():
        if element in ratings.loc[ratings.movieId==movie2, 'userId'].tolist():
            rating1.append(get_rating(element,movie1))
            rating2.append(get_rating(element,movie2))

    return dot(rating1, rating2)/(norm(rating1)*norm(rating2))

### get K item nearest
def most_similar_movie(movie1, K_number_user, similarity_name):
  movieid = ratings.movieId.unique().tolist()
  # print(len(movieid))

  if similarity_name == 'pearson':
    similarity_score =  [(pearson_correlation_score_Item(movie1, movie_i), movie_i) for movie_i in movieid if movie_i != movie1]

  if similarity_name == 'cosine':
    similarity_score =  [(distance_similarity_score_Item(movie1, movie_i), movie_i) for movie_i in movieid if movie_i != movie1]

  similarity_score.sort() # ascending
  similarity_score.reverse() # descending

  return similarity_score[:K_number_user] # can change K user nearest


def get_userids(movieid):
  return (ratings.loc[(ratings.movieId==movieid), 'userId'].tolist())

# get recommendation for 1 user based on Aggregate weighted ratings
def rating_predict_based_on_item(movieid, K_number_item, sim_name):

  total = {}
  sum_similarity = {}
  list_movie_popular = most_similar_movie(movieid, K_number_item, sim_name)

  for similarityName, item_pop in list_movie_popular: # lặp từng user trong list popular
    score = similarityName
    userids = get_userids(item_pop)

    for userid in userids:
      if userid not in get_movieids_seen(movieid):
        if userid not in total:
         total[userid] = 0
         sum_similarity[userid] = 0

        total[userid] += get_rating(userid, item_pop)*score
        sum_similarity[userid] += score

    # calculate rating according to Aggregate weighted ratings
    list_ranking = []
    for movieid,tot in total.items():
     if sum_similarity[movieid] == 0:
       list_ranking.append((0,movieid))
     else:
       rating = tot/(sum_similarity[movieid])
       list_ranking.append((rating,movieid))

  list_ranking = [(userid, tot/sum_similarity[userid]) for userid, tot in total.items()]

  return list_ranking

### get name + the high rating predictions of topK=10 items
def get_recommendation_based_on_item(movieid, K_number_user, similarity_name, topN = 10):

  list_ranking = rating_predict_based_on_item(movieid, K_number_user, similarity_name)

  list_ranking.sort()
  list_ranking.reverse()

  return list_ranking[:topN]

## Predicting on test set

## User

In [ ]:
# hàm dữ đoán trên tập test
def get_recommendation_on_test_user(test):
  X_test_pred = []
  X_test_userid = test['userId'].tolist()   #lấy ra danh sách các userID trong tập test
  X_test_movieID = test['movieId'].tolist()

  print("------Dự đoán cho tập test với {} điểm dữ liệu------".format(len(X_test_userid)))

  for i in range(len(X_test_userid)):        #lấy ra từng user
    list_R = get_recommendation_based_on_user(X_test_userid[i], 200, 'pearson', 10)    # lấy ra danh sách khuyến nghị dựa trên USER

    check = 0
    for j in list_R:
      if(X_test_movieID[i] == j[0]):     #j[1] là lấy movieID predict
        X_test_pred.append(j[1])
        check = 1
    if(check == 0):
      X_test_pred.append(0)
    print('----- Đang dự đoán cho dòng thứ {}'.format(i), 'là {}'.format(X_test_pred[i]))
  return X_test_pred

In [ ]:
# hàm dữ đoán trên tập test
def get_recommendation_on_test_user_2(test):
  X_test_pred = []
  X_test_userid = test['userId'].tolist()   #lấy ra danh sách các userID trong tập test
  X_test_movieID = test['movieId'].tolist()

  print("------Dự đoán cho tập test với {} điểm dữ liệu------".format(len(X_test_userid)))

  for i in range(len(X_test_userid)):        #lấy ra từng user
    list_R = get_recommendation_based_on_user(X_test_userid[i], 200, 'cosine', 10)    # lấy ra danh sách khuyến nghị dựa trên USER

    check = 0
    for j in list_R:
      if(X_test_movieID[i] == j[0]):     #j[1] là lấy movieID predict
        X_test_pred.append(j[1])
        check = 1
    if(check == 0):
      X_test_pred.append(0)
    print('----- Đang dự đoán cho dòng thứ {}'.format(i), 'là {}'.format(X_test_pred[i]))
  return X_test_pred

In [ ]:
start = time.time()
X_test_pred_user_pearson = get_recommendation_on_test_user(X_test)
X_test_pred_user_cosine = get_recommendation_on_test_user_2(X_test)
end = time.time()
print(f'Tổng thời gian dự đoán: {(end - start)/60} minutes')

In [ ]:
X_test['X_test_pred_user_pearson'] = X_test_pred_user_pearson
X_test['X_test_pred_user_cosine'] = X_test_pred_user_cosine

In [ ]:
X_test

,userId,movieId,rating,timestamp
1621,5932,307,10.0,1.458000e+09
2605,2542,412,10.0,1.485994e+09
4890,944,301,6.0,1.543363e+09
2040,5857,228,8.0,1.472083e+09
5399,5885,139,6.6,1.553904e+09
...,...,...,...,...
5365,5972,210,9.0,1.552867e+09
15687,5914,394,8.8,1.679098e+09
1896,5961,128,6.0,1.468109e+09
6694,6786,65,6.3,1.578442e+09


### Save prediction results

In [ ]:
X_test.to_csv('X_test_UserKNN.csv')

## Item

In [ ]:
# Hàm dữ đoán trên tập test
def get_recommendation_on_test_item(test):
  X_test_pred = []
  X_test_userid = test['userId'].tolist()   #lấy ra danh sách các userID trong tập test
  X_test_movieID = test['movieId'].tolist()

  print("------Dự đoán cho tập test với {} điểm dữ liệu------".format(len(X_test_movieID)))

  for i in range(len(X_test_movieID)):        #lấy ra từng user
    list_R = get_recommendation_based_on_item(X_test_movieID[i], 100, 'pearson', 10)    # lấy ra danh sách khuyến nghị dựa trên USER

    check = 0
    for j in list_R:
      if(X_test_userid[i] == j[0]):     #j[1] là lấy movieID predict
        X_test_pred.append(j[1])
        check = 1
    if(check == 0):
      X_test_pred.append(0)
    print('----- Đang dự đoán cho dòng thứ {}'.format(i), 'là {}'.format(X_test_pred[i]))
  return X_test_pred

In [ ]:
start = time.time()
X_test_pred_item_pearson = get_recommendation_on_test_item(X_test)
end = time.time()
print(f'Tổng thời gian dự đoán: {(end - start)/60} minutes')

------Dự đoán cho tập test với 959 điểm dữ liệu------


<ipython-input-31-245d4e089628>:99: RuntimeWarning: invalid value encountered in double_scalars
  list_ranking = [(userid, tot/sum_similarity[userid]) for userid, tot in total.items()]


----- Đang dự đoán cho dòng thứ 0 là 7.6825269330104975
----- Đang dự đoán cho dòng thứ 1 là 7.29664344095203
----- Đang dự đoán cho dòng thứ 2 là 7.0
----- Đang dự đoán cho dòng thứ 3 là 6.31283513209117
----- Đang dự đoán cho dòng thứ 4 là 0
----- Đang dự đoán cho dòng thứ 5 là 7.397645858722833
----- Đang dự đoán cho dòng thứ 6 là 8.5
----- Đang dự đoán cho dòng thứ 7 là 4.0
----- Đang dự đoán cho dòng thứ 8 là 6.375123857048104
----- Đang dự đoán cho dòng thứ 9 là 9.0
----- Đang dự đoán cho dòng thứ 10 là 5.836958959306481
----- Đang dự đoán cho dòng thứ 11 là 7.0
----- Đang dự đoán cho dòng thứ 12 là 7.39352004450547
----- Đang dự đoán cho dòng thứ 13 là 6.397991744848035
----- Đang dự đoán cho dòng thứ 14 là 8.892558373830564
----- Đang dự đoán cho dòng thứ 15 là 6.0
----- Đang dự đoán cho dòng thứ 16 là 7.3890400363086846
----- Đang dự đoán cho dòng thứ 17 là nan
----- Đang dự đoán cho dòng thứ 18 là 7.569921391704029
----- Đang dự đoán cho dòng thứ 19 là 9.516154497425578
-----

In [ ]:
# hàm dữ đoán trên tập test
def get_recommendation_on_test_item_2(test):
  X_test_pred = []
  X_test_userid = test['userId'].tolist()   #lấy ra danh sách các userID trong tập test
  X_test_movieID = test['movieId'].tolist()

  print("------Dự đoán cho tập test với {} điểm dữ liệu------".format(len(X_test_movieID)))

  for i in range(len(X_test_movieID)):        #lấy ra từng user
    list_R = get_recommendation_based_on_item(X_test_movieID[i], 100, 'cosine', 10)    # lấy ra danh sách khuyến nghị dựa trên USER

    check = 0
    for j in list_R:
      if(X_test_userid[i] == j[0]):     #j[1] là lấy movieID predict
        X_test_pred.append(j[1])
        check = 1
    if(check == 0):
      X_test_pred.append(0)
    print('----- Đang dự đoán cho dòng thứ {}'.format(i), 'là {}'.format(X_test_pred[i]))
  return X_test_pred

In [ ]:
start = time.time()
X_test_pred_item_cosine = get_recommendation_on_test_item_2(X_test)
end = time.time()
print(f'Tổng thời gian dự đoán: {(end - start)/60} minutes')

------Dự đoán cho tập test với 959 điểm dữ liệu------
----- Đang dự đoán cho dòng thứ 0 là 8.338461538461539
----- Đang dự đoán cho dòng thứ 1 là 0
----- Đang dự đoán cho dòng thứ 2 là 7.875
----- Đang dự đoán cho dòng thứ 3 là 8.014583333333334
----- Đang dự đoán cho dòng thứ 4 là 8.500943574561786
----- Đang dự đoán cho dòng thứ 5 là 7.5
----- Đang dự đoán cho dòng thứ 6 là 8.6


<ipython-input-31-245d4e089628>:99: RuntimeWarning: invalid value encountered in double_scalars
  list_ranking = [(userid, tot/sum_similarity[userid]) for userid, tot in total.items()]


----- Đang dự đoán cho dòng thứ 7 là 6.706985690586417
----- Đang dự đoán cho dòng thứ 8 là 8.049999999999999
----- Đang dự đoán cho dòng thứ 9 là 7.099522185236129
----- Đang dự đoán cho dòng thứ 10 là 6.15
----- Đang dự đoán cho dòng thứ 11 là 8.0
----- Đang dự đoán cho dòng thứ 12 là 6.8
----- Đang dự đoán cho dòng thứ 13 là 9.4
----- Đang dự đoán cho dòng thứ 14 là 8.5
----- Đang dự đoán cho dòng thứ 15 là 7.763250493090887
----- Đang dự đoán cho dòng thứ 16 là 6.0
----- Đang dự đoán cho dòng thứ 17 là 8.315094339622641
----- Đang dự đoán cho dòng thứ 18 là 7.466666666666666
----- Đang dự đoán cho dòng thứ 19 là 8.0
----- Đang dự đoán cho dòng thứ 20 là 7.170161962151357
----- Đang dự đoán cho dòng thứ 21 là 0
----- Đang dự đoán cho dòng thứ 22 là 6.956525581083903
----- Đang dự đoán cho dòng thứ 23 là 10.0
----- Đang dự đoán cho dòng thứ 24 là 7.12273345687932
----- Đang dự đoán cho dòng thứ 25 là 0
----- Đang dự đoán cho dòng thứ 26 là 7.733333333333333
----- Đang dự đoán cho dòn

In [ ]:
X_test['X_test_pred_item_pearson'] = X_test_pred_item_pearson
X_test['X_test_pred_item_cosine'] = X_test_pred_item_cosine

### Save prediction results

In [ ]:
X_test.to_csv('X_test_ItemKNN.csv')

## Evaluation

In [90]:
df_UserKNN = pd.read_csv('/content/X_test_UserKNN.csv')
df_UserKNN

,Unnamed: 0,userId,movieId,rating,timestamp,X_test_pred_user_pearson,X_test_pred_user_cosine
0,1621,5932,307,10.0,1.458000e+09,NaN,7.643336
1,2605,2542,412,10.0,1.485994e+09,9.392191,8.802912
2,4890,944,301,6.0,1.543363e+09,NaN,7.627967
3,2040,5857,228,8.0,1.472083e+09,9.000000,0.000000
4,5399,5885,139,6.6,1.553904e+09,0.000000,7.306552
...,...,...,...,...,...,...,...
954,5365,5972,210,9.0,1.552867e+09,9.000000,9.500318
955,15687,5914,394,8.8,1.679098e+09,9.178430,9.300489
956,1896,5961,128,6.0,1.468109e+09,6.262999,0.000000
957,6694,6786,65,6.3,1.578442e+09,7.000000,7.399552


In [91]:
df_UserKNN['X_test_pred_user_pearson'] = df_UserKNN['X_test_pred_user_pearson'].fillna(0)
df_UserKNN

,Unnamed: 0,userId,movieId,rating,timestamp,X_test_pred_user_pearson,X_test_pred_user_cosine
0,1621,5932,307,10.0,1.458000e+09,0.000000,7.643336
1,2605,2542,412,10.0,1.485994e+09,9.392191,8.802912
2,4890,944,301,6.0,1.543363e+09,0.000000,7.627967
3,2040,5857,228,8.0,1.472083e+09,9.000000,0.000000
4,5399,5885,139,6.6,1.553904e+09,0.000000,7.306552
...,...,...,...,...,...,...,...
954,5365,5972,210,9.0,1.552867e+09,9.000000,9.500318
955,15687,5914,394,8.8,1.679098e+09,9.178430,9.300489
956,1896,5961,128,6.0,1.468109e+09,6.262999,0.000000
957,6694,6786,65,6.3,1.578442e+09,7.000000,7.399552


In [92]:
test_df = pd.DataFrame(data=df_UserKNN[['userId',	'movieId','rating']])
test_df = test_df.sort_values(['userId', 'rating'], ascending=True)
test_df

,userId,movieId,rating
446,70,402,7.0
381,100,395,1.0
614,100,425,6.0
740,100,465,8.0
800,126,220,6.9
...,...,...,...
54,10594,404,5.7
162,10636,65,10.0
186,10636,253,10.0
725,10657,399,10.0


In [93]:
top_k_df_UserKNN_pearson = pd.DataFrame(data=df_UserKNN[['userId',	'movieId', 'X_test_pred_user_pearson']])
top_k_df_UserKNN_pearson = top_k_df_UserKNN_pearson[top_k_df_UserKNN_pearson.X_test_pred_user_pearson > 7.5]
top_k_df_UserKNN_pearson = top_k_df_UserKNN_pearson.sort_values(['userId', 'X_test_pred_user_pearson'], ascending=True)
top_k_df_UserKNN_pearson

,userId,movieId,X_test_pred_user_pearson
740,100,465,10.000000
800,126,220,7.655451
229,145,381,8.000000
819,145,426,10.000000
814,463,420,8.950000
...,...,...,...
915,10249,561,9.587509
873,10254,430,8.407950
526,10266,412,9.467005
364,10516,222,8.232221


In [94]:
from recommenders.evaluation.python_evaluation import auc, map_at_k, ndcg_at_k, precision_at_k, recall_at_k
def ranking_metrics(
    data_true,
    data_pred,
    K
):

    eval_map = map_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                    col_rating="rating", col_prediction="X_test_pred_user_pearson",
                    relevancy_method="top_k", k= K)

    eval_ndcg_5 = ndcg_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                      col_rating="rating", col_prediction="X_test_pred_user_pearson",
                      relevancy_method="top_k", k= 5)

    eval_ndcg_10 = ndcg_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                      col_rating="rating", col_prediction="X_test_pred_user_pearson",
                      relevancy_method="top_k", k= K)

    eval_precision_10 = precision_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                               col_rating="rating", col_prediction="X_test_pred_user_pearson",
                               relevancy_method="top_k", k= K)

    eval_precision_5 = precision_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                               col_rating="rating", col_prediction="X_test_pred_user_pearson",
                               relevancy_method="top_k", k= 5)

    eval_recall_10 = recall_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                          col_rating="rating", col_prediction="X_test_pred_user_pearson",
                          relevancy_method="top_k", k= K)

    eval_recall_5 = recall_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                          col_rating="rating", col_prediction="X_test_pred_user_pearson",
                          relevancy_method="top_k", k= 5)


    df_result = pd.DataFrame(
        {
            "MAP": eval_map,
            "nDCG@10": eval_ndcg_10,
            "nDCG@5": eval_ndcg_5,
            "Precision@10": eval_precision_10,
            "Precision@5": eval_precision_5,
            "Recall@10": eval_recall_10,
            "Recall@5": eval_recall_5,
        },
        index=[0]
    )

    return df_result

In [95]:
df = df.rename(columns={'IDuser': "userId", 'IDhotel': "movieId", 'Rating': "rating"})

### top_k_df_UserKNN_pearson

In [96]:
eval = ranking_metrics(
    data_true=df,
    data_pred=top_k_df_UserKNN_pearson,
    K=10
)

eval

,MAP,nDCG@10,nDCG@5,Precision@10,Precision@5,Recall@10,Recall@5
0,0.192579,0.010229,0.013837,0.192609,0.362609,0.178047,0.176776


In [97]:
top_k_df_UserKNN_cosine = pd.DataFrame(data=df_UserKNN[['userId',	'movieId', 'X_test_pred_user_cosine']])
top_k_df_UserKNN_cosine = top_k_df_UserKNN_cosine[top_k_df_UserKNN_cosine.X_test_pred_user_cosine > 7.5]
top_k_df_UserKNN_cosine = top_k_df_UserKNN_cosine.sort_values(['userId', 'X_test_pred_user_cosine'], ascending=True)
top_k_df_UserKNN_cosine

,userId,movieId,X_test_pred_user_cosine
446,70,402,8.444917
740,100,465,7.615357
800,126,220,7.902097
933,415,515,7.583615
814,463,420,7.659743
...,...,...,...
873,10254,430,7.676133
526,10266,412,8.835199
364,10516,222,7.579740
768,10516,383,8.987949


In [98]:
from recommenders.evaluation.python_evaluation import auc, map_at_k, ndcg_at_k, precision_at_k, recall_at_k
def ranking_metrics(
    data_true,
    data_pred,
    K
):

    eval_map = map_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                    col_rating="rating", col_prediction="X_test_pred_user_cosine",
                    relevancy_method="top_k", k= K)

    eval_ndcg_5 = ndcg_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                      col_rating="rating", col_prediction="X_test_pred_user_cosine",
                      relevancy_method="top_k", k= 5)

    eval_ndcg_10 = ndcg_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                      col_rating="rating", col_prediction="X_test_pred_user_cosine",
                      relevancy_method="top_k", k= K)

    eval_precision_10 = precision_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                               col_rating="rating", col_prediction="X_test_pred_user_cosine",
                               relevancy_method="top_k", k= K)

    eval_precision_5 = precision_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                               col_rating="rating", col_prediction="X_test_pred_user_cosine",
                               relevancy_method="top_k", k= 5)

    eval_recall_10 = recall_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                          col_rating="rating", col_prediction="X_test_pred_user_cosine",
                          relevancy_method="top_k", k= K)

    eval_recall_5 = recall_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                          col_rating="rating", col_prediction="X_test_pred_user_cosine",
                          relevancy_method="top_k", k= 5)

    df_result = pd.DataFrame(
        {
            "MAP": eval_map,
            "nDCG@10": eval_ndcg_10,
            "nDCG@5": eval_ndcg_5,
            "Precision@10": eval_precision_10,
            "Precision@5": eval_precision_5,
            "Recall@10": eval_recall_10,
            "Recall@5": eval_recall_5,
        },
        index=[0]
    )

    return df_result

### top_k_df_UserKNN_cosine

In [99]:
eval = ranking_metrics(
    data_true=df,
    data_pred=top_k_df_UserKNN_cosine,
    K=10
)

eval

,MAP,nDCG@10,nDCG@5,Precision@10,Precision@5,Recall@10,Recall@5
0,0.215113,0.013535,0.017615,0.195804,0.367832,0.198342,0.197116


In [100]:
df_ItemKNN = pd.read_csv('/content/X_test_ItemKNN.csv')
df_ItemKNN

,Unnamed: 0,userId,movieId,rating,timestamp,X_test_pred_item_pearson,X_test_pred_item_cosine
0,1621,5932,307,10.0,1.458000e+09,7.682527,8.338462
1,2605,2542,412,10.0,1.485994e+09,7.296643,0.000000
2,4890,944,301,6.0,1.543363e+09,7.000000,7.875000
3,2040,5857,228,8.0,1.472083e+09,6.312835,8.014583
4,5399,5885,139,6.6,1.553904e+09,0.000000,8.500944
...,...,...,...,...,...,...,...
954,5365,5972,210,9.0,1.552867e+09,8.125247,7.833333
955,15687,5914,394,8.8,1.679098e+09,8.799790,7.531034
956,1896,5961,128,6.0,1.468109e+09,9.000000,8.079964
957,6694,6786,65,6.3,1.578442e+09,NaN,7.498271


In [101]:
df_ItemKNN['X_test_pred_item_pearson'] = df_ItemKNN['X_test_pred_item_pearson'].fillna(0)
df_ItemKNN

,Unnamed: 0,userId,movieId,rating,timestamp,X_test_pred_item_pearson,X_test_pred_item_cosine
0,1621,5932,307,10.0,1.458000e+09,7.682527,8.338462
1,2605,2542,412,10.0,1.485994e+09,7.296643,0.000000
2,4890,944,301,6.0,1.543363e+09,7.000000,7.875000
3,2040,5857,228,8.0,1.472083e+09,6.312835,8.014583
4,5399,5885,139,6.6,1.553904e+09,0.000000,8.500944
...,...,...,...,...,...,...,...
954,5365,5972,210,9.0,1.552867e+09,8.125247,7.833333
955,15687,5914,394,8.8,1.679098e+09,8.799790,7.531034
956,1896,5961,128,6.0,1.468109e+09,9.000000,8.079964
957,6694,6786,65,6.3,1.578442e+09,0.000000,7.498271


In [102]:
top_k_df_ItemKNN_pearson = pd.DataFrame(data=df_ItemKNN[['userId',	'movieId', 'X_test_pred_item_pearson']])
top_k_df_ItemKNN_pearson = top_k_df_ItemKNN_pearson[top_k_df_ItemKNN_pearson.X_test_pred_item_pearson > 7.5]
top_k_df_ItemKNN_pearson = top_k_df_ItemKNN_pearson.sort_values(['userId', 'X_test_pred_item_pearson'], ascending=True)
top_k_df_ItemKNN_pearson

,userId,movieId,X_test_pred_item_pearson
229,145,381,9.341011
933,415,515,9.000000
83,463,112,8.000000
814,463,420,8.750000
372,463,178,9.700000
...,...,...,...
839,10260,124,10.000000
141,10516,573,8.348842
54,10594,404,8.500000
162,10636,65,8.092978


In [103]:
from recommenders.evaluation.python_evaluation import auc, map_at_k, ndcg_at_k, precision_at_k, recall_at_k
def ranking_metrics(
    data_true,
    data_pred,
    K
):

    eval_map = map_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                    col_rating="rating", col_prediction="X_test_pred_item_pearson",
                    relevancy_method="top_k", k= K)

    eval_ndcg_5 = ndcg_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                      col_rating="rating", col_prediction="X_test_pred_item_pearson",
                      relevancy_method="top_k", k= 5)

    eval_ndcg_10 = ndcg_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                      col_rating="rating", col_prediction="X_test_pred_item_pearson",
                      relevancy_method="top_k", k= K)

    eval_precision_10 = precision_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                               col_rating="rating", col_prediction="X_test_pred_item_pearson",
                               relevancy_method="top_k", k= K)

    eval_precision_5 = precision_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                               col_rating="rating", col_prediction="X_test_pred_item_pearson",
                               relevancy_method="top_k", k= 5)

    eval_recall_10 = recall_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                          col_rating="rating", col_prediction="X_test_pred_item_pearson",
                          relevancy_method="top_k", k= K)

    eval_recall_5 = recall_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                          col_rating="rating", col_prediction="X_test_pred_item_pearson",
                          relevancy_method="top_k", k= 5)

    df_result = pd.DataFrame(
        {
            "MAP": eval_map,
            "nDCG@10": eval_ndcg_10,
            "nDCG@5": eval_ndcg_5,
            "Precision@10": eval_precision_10,
            "Precision@5": eval_precision_5,
            "Recall@10": eval_recall_10,
            "Recall@5": eval_recall_5,
        },
        index=[0]
    )

    return df_result

### top_k_df_ItemKNN_pearson

In [104]:
eval = ranking_metrics(
    data_true=df,
    data_pred=top_k_df_ItemKNN_pearson,
    K=10
)

eval

,MAP,nDCG@10,nDCG@5,Precision@10,Precision@5,Recall@10,Recall@5
0,0.206581,0.010109,0.013637,0.212195,0.390244,0.185897,0.183694


In [105]:
top_k_df_ItemKNN_cosine = pd.DataFrame(data=df_ItemKNN[['userId',	'movieId', 'X_test_pred_item_cosine']])
top_k_df_ItemKNN_cosine = top_k_df_ItemKNN_cosine[top_k_df_ItemKNN_cosine.X_test_pred_item_cosine > 7.5]
top_k_df_ItemKNN_cosine = top_k_df_ItemKNN_cosine.sort_values(['userId', 'X_test_pred_item_cosine'], ascending=True)
top_k_df_ItemKNN_cosine

,userId,movieId,X_test_pred_item_cosine
800,126,220,8.500000
819,145,426,7.600000
933,415,515,7.666667
148,415,206,8.000000
372,463,178,9.000000
...,...,...,...
768,10516,383,8.360000
364,10516,222,8.397775
141,10516,573,8.700000
186,10636,253,8.316667


In [106]:
from recommenders.evaluation.python_evaluation import auc, map_at_k, ndcg_at_k, precision_at_k, recall_at_k
def ranking_metrics(
    data_true,
    data_pred,
    K
):

    eval_map = map_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                    col_rating="rating", col_prediction="X_test_pred_item_cosine",
                    relevancy_method="top_k", k= K)

    eval_ndcg_5 = ndcg_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                      col_rating="rating", col_prediction="X_test_pred_item_cosine",
                      relevancy_method="top_k", k= 5)

    eval_ndcg_10 = ndcg_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                      col_rating="rating", col_prediction="X_test_pred_item_cosine",
                      relevancy_method="top_k", k= K)

    eval_precision_10 = precision_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                               col_rating="rating", col_prediction="X_test_pred_item_cosine",
                               relevancy_method="top_k", k= K)

    eval_precision_5 = precision_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                               col_rating="rating", col_prediction="X_test_pred_item_cosine",
                               relevancy_method="top_k", k= 5)

    eval_recall_10 = recall_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                          col_rating="rating", col_prediction="X_test_pred_item_cosine",
                          relevancy_method="top_k", k= K)

    eval_recall_5 = recall_at_k(data_true, data_pred, col_user="userId", col_item="movieId",
                          col_rating="rating", col_prediction="X_test_pred_item_cosine",
                          relevancy_method="top_k", k= 5)

    df_result = pd.DataFrame(
        {
            "MAP": eval_map,
            "nDCG@10": eval_ndcg_10,
            "nDCG@5": eval_ndcg_5,
            "Precision@10": eval_precision_10,
            "Precision@5": eval_precision_5,
            "Recall@10": eval_recall_10,
            "Recall@5": eval_recall_5,
        },
        index=[0]
    )

    return df_result

### top_k_df_ItemKNN_cosine

In [107]:
eval = ranking_metrics(
    data_true=df,
    data_pred=top_k_df_ItemKNN_cosine,
    K=10
)

eval

,MAP,nDCG@10,nDCG@5,Precision@10,Precision@5,Recall@10,Recall@5
0,0.222896,0.012024,0.015886,0.215745,0.394043,0.200442,0.197538
